## 启动服务

In [1]:
import mag
import time
import json
from IPython.display import display, Markdown,HTML
import os
import webbrowser
import subprocess

In [2]:
# 启动服务
mag.start()

MAG服务器已启动 (PID: 15408), 日志: C:\Users\keta\AppData\Local\Temp\mag_sdk_logs\mag_server.log


True

In [1]:
# 关闭服务
mag.shutdown()

NameError: name 'mag' is not defined

In [3]:
is_running = mag.is_running()
print(f"服务运行状态: {is_running}")

服务运行状态: True


## 模型管理

#### （1）新增模型

In [ ]:
new_model = {
    "name": "deepseek-chat",
    "base_url": "https://gateway.ai.cloudflare.com/v1/79dc4e1b350a1ac66b63c0f6a9e0c7a6/deepseek/",
    "api_key": "sk-f6f25d0f12ce41d5b52ae9d45e14005a",
    "model": "deepseek-chat"
}
mag.add_model(new_model)

#### （2）列出模型

In [4]:
models = mag.list_models()
print(f"找到 {len(models)} 个模型")
for model in models:
    print(f"\n模型名称：{model['name']} \nBase_url：{model['base_url']}\n")

找到 2 个模型

模型名称：Qwen/Qwen2.5-32B-Instruct 
Base_url：https://api-inference.modelscope.cn/v1/


模型名称：deepseek-chat 
Base_url：https://api.deepseek.com



#### （3）更新模型

In [ ]:
# 更新模型
updated_model = {
    "name": "deepseek-chat",
    "base_url": "https://gateway.ai.cloudflare.com/v1/79dc4e1b350a1ac66b63c0f6a9e0c7a6/deepseek/",
    "api_key": "sk-f6f25d0f12ce41d5b52ae9d45e14005a",
    "model": "deepseek-chat"
}
result = mag.update_model("deepseek-chat", updated_model)
print(result)

#### （4）删除模型

In [ ]:
mag.delete_model("deepseek-chat")

## MCP管理

#### （1）增加/删除server

In [ ]:
# 增加server
servers = {"mcpServers": {
    "fetch": {
      "autoApprove": [
        "fetch"
      ],
      "timeout": 60,
      "command": "uvx",
      "args": [
        "mcp-server-fetch"
      ],
      "transportType": "stdio"
    },
    "memory": {
      "autoApprove": [],
      "timeout": 60,
      "command": "docker",
      "args": [
        "run",
        "-i",
        "-v",
        "claude-memory:/app/dist",
        "--rm",
        "mcp/memory"
      ],
      "transportType": "stdio",
    }
}}
result = mag.add_server(servers=servers)
print(json.dumps(result, indent=4, ensure_ascii=False))

In [ ]:
# 删除server
result = mag.remove_server(["fetch","memory"])
print(json.dumps(result, indent=4, ensure_ascii=False))

#### （2）更新server配置文件（适合全面更新）

In [ ]:
# 更新mcp server
# 此举将会替代之前的配置文件
config={
  "mcpServers": {
    "memory": {
      "autoApprove": [],
      "timeout": 60,
      "command": "docker",
      "args": [
        "run",
        "-i",
        "-v",
        "claude-memory:/app/dist",
        "--rm",
        "mcp/memory"
      ],
      "transportType": "stdio",
    },
    "fetch": {
      "autoApprove": [
        "fetch"
      ],
      "timeout": 60,
      "command": "uvx",
      "args": [
        "mcp-server-fetch"
      ],
      "transportType": "stdio",
    },
    "sequentialthinking": {
      "command": "docker",
      "args": [
        "run",
        "--rm",
        "-i",
        "mcp/sequentialthinking"
      ]
    },
    "tavily-mcp": {
      "command": "npx",
      "args": ["-y", "tavily-mcp@0.1.2"],
      "env": {
        "TAVILY_API_KEY": "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"
      }
    }
  }
}
result = mag.update_mcp_config(config=config)
print(json.dumps(result, indent=4, ensure_ascii=False))

{
    "status": {
        "message": "配置已更新并通知MCP Client"
    }
}


#### （3）连接mcp server

In [6]:
# 连接单个
mag.connect_mcp("tavily-mcp")

{'status': 'connected',
 'server': 'tavily-mcp',
 'tools': ['tavily-search', 'tavily-extract']}

In [7]:
# 连接所有
result = mag.connect_mcp("all")
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "status": "completed",
  "summary": {
    "total_servers": 4,
    "successful_connections": 0,
    "failed_connections": 0,
    "already_connected": 4
  },
  "servers": {
    "memory": {
      "status": "already_connected",
      "tools": [
        "create_entities",
        "create_relations",
        "add_observations",
        "delete_entities",
        "delete_observations",
        "delete_relations",
        "read_graph",
        "search_nodes",
        "open_nodes"
      ]
    },
    "fetch": {
      "status": "already_connected",
      "tools": [
        "fetch"
      ]
    },
    "sequentialthinking": {
      "status": "already_connected",
      "tools": [
        "sequentialthinking"
      ]
    },
    "tavily-mcp": {
      "status": "already_connected",
      "tools": [
        "tavily-search",
        "tavily-extract"
      ]
    }
  },
  "tools": {
    "memory": [
      "create_entities",
      "create_relations",
      "add_observations",
      "delete_entities",
    

#### （4）获取MCP配置

In [ ]:
mcp_config = mag.get_mcp_config()
print(json.dumps(mcp_config, indent=8, ensure_ascii=False))

#### （6）获取MCP状态/查看工具

In [6]:
mcp_status = mag.get_mcp_status()
for server, status in mcp_status.items():
    print(f"\n{server}: {'已连接' if status.get('connected') else '未连接'}")
    print(f"{server}工具：{status.get('tools')}")
print(json.dumps(mcp_status, indent=4, ensure_ascii=False))


memory: 已连接
memory工具：['create_entities', 'create_relations', 'add_observations', 'delete_entities', 'delete_observations', 'delete_relations', 'read_graph', 'search_nodes', 'open_nodes']

fetch: 已连接
fetch工具：['fetch']

sequentialthinking: 已连接
sequentialthinking工具：['sequentialthinking']

tavily-mcp: 已连接
tavily-mcp工具：['tavily-search', 'tavily-extract']
{
    "memory": {
        "connected": true,
        "init_attempted": true,
        "tools": [
            "create_entities",
            "create_relations",
            "add_observations",
            "delete_entities",
            "delete_observations",
            "delete_relations",
            "read_graph",
            "search_nodes",
            "open_nodes"
        ],
        "error": null
    },
    "fetch": {
        "connected": true,
        "init_attempted": true,
        "tools": [
            "fetch"
        ],
        "error": null
    },
    "sequentialthinking": {
        "connected": true,
        "init_attempted": true,

#### （7）获取mcp工具参数

In [5]:
mcp_tools = mag.get_tools()
print(json.dumps(mcp_tools, indent=2, ensure_ascii=False))

{
  "memory": [
    {
      "name": "create_entities",
      "description": "Create multiple new entities in the knowledge graph",
      "input_schema": {
        "type": "object",
        "properties": {
          "entities": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "name": {
                  "type": "string",
                  "description": "The name of the entity"
                },
                "entityType": {
                  "type": "string",
                  "description": "The type of the entity"
                },
                "observations": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  },
                  "description": "An array of observation contents associated with the entity"
                }
              },
              "required": [
                "name",
                "entityType",
     

## 图管理

#### （1）导入一个图

In [4]:
# 从包导入
# graph = mag.import_graph("easy_search.zip")
# print(json.dumps(graph, indent=2, ensure_ascii=False))

# 从config导入
graph = mag.import_graph(r"deepsearch.json")
print(json.dumps(graph, indent=2, ensure_ascii=False))

{
  "status": "success",
  "message": "图 'deepresearch' 导入成功"
}


#### （2）导出一个图

In [ ]:
graph = mag.export_graph("news_processing_workflow")
print(json.dumps(graph, indent=2, ensure_ascii=False))

#### （3）列出所有图

In [8]:
print("\n列出所有图:")
graphs = mag.list_graphs()
print(f"找到 {len(graphs)} 个图: {graphs}")


列出所有图:
找到 4 个图: ['Bilibili搜索循环测试', 'deepresearch', 'news_processing_workflow', 'news_report']


#### （4）查看一个图配置文件

In [9]:
result = mag.get_graph('news_processing_workflow')
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "name": "news_processing_workflow",
  "description": "新闻抓取与多格式总结处理流程",
  "nodes": [
    {
      "name": "fetch_news_1",
      "description": "抓取第一篇新闻",
      "model_name": "deepseek-chat",
      "mcp_servers": [
        "fetch"
      ],
      "system_prompt": "",
      "user_prompt": "https://news.qq.com/rain/a/20250503A06K0600 总结这篇新闻",
      "input_nodes": [
        "start"
      ],
      "output_nodes": [
        "news_broadcast_script",
        "news_summary_html"
      ],
      "handoffs": null,
      "global_output": true,
      "context": [],
      "context_mode": "all",
      "context_n": 1,
      "is_start": true,
      "is_end": false,
      "output_enabled": true,
      "is_subgraph": false,
      "subgraph_name": null,
      "level": 0,
      "save": null
    },
    {
      "name": "fetch_news_2",
      "description": "抓取第二篇新闻",
      "model_name": "deepseek-chat",
      "mcp_servers": [
        "fetch"
      ],
      "system_prompt": "",
      "user_prompt": "https://ne

#### （5）绘图

In [8]:
graph_detail = mag.get_graph_detail("deepresearch")

# 渲染README
if graph_detail['readme']:
    display(Markdown(graph_detail['readme']))
else:
    print("未找到README文件")

# deepresearch
深度分析用户问题，进行多轮智能检索，并最终生成可视化HTML网页的综合研究系统

## 流程图
```mermaid
graph TD
    Input(["start"]);
    Input(["start"]) --> question_analyzer["question_analyzer"];
    question_analyzer["question_analyzer"] --> todo_maker["todo_maker"];
    todo_maker["todo_maker"] --> search_executor["search_executor"];
    search_executor["search_executor"] --> knowledge_manager["knowledge_manager"];
    knowledge_manager["knowledge_manager"] --> progress_checker{"progress_checker"};
    progress_checker{"progress_checker"} --> search_executor["search_executor"];
    progress_checker{"progress_checker"} --> webpage_generator["webpage_generator"];
    webpage_generator["webpage_generator"] --> Output(["end"]);
```

## 节点列表
### question_analyzer
- 描述: 深度分析用户提出的问题，识别关键概念、研究维度和潜在需求
- 使用模型: deepseek-chat

### todo_maker
- 描述: 基于问题分析结果，制作详细的研究TODO清单，指导后续检索方向
- 使用模型: deepseek-chat

### search_executor
- 描述: 执行网络搜索检索，获取相关信息内容
- 使用模型: deepseek-chat
- MCP服务器: tavily-mcp, fetch

### knowledge_manager
- 描述: 管理和存储检索到的知识内容，构建结构化知识库
- 使用模型: deepseek-chat
- MCP服务器: memory

### progress_checker
- 描述: 检查研究进度，决定是否继续检索或开始网页生成
- 使用模型: deepseek-chat
- MCP服务器: memory
- 最大决策次数: 5

### webpage_generator
- 描述: 根据研究结果生成可视化HTML网页并保存
- 使用模型: deepseek-chat
- MCP服务器: memory

## 使用的MCP服务器
### fetch
- 状态: 启用
- 超时: 60秒
- 自动批准的工具: fetch

### memory
- 状态: 启用
- 超时: 60秒

### tavily-mcp
- 状态: 启用
- 超时: 60秒

## 使用的模型
### deepseek-chat
- 基础URL: https://gateway.ai.cloudflare.com/v1/79dc4e1b350a1ac66b63c0f6a9e0c7a6/deepseek/
- 模型标识符: deepseek-chat


#### （6）将图导出为mcp server 脚本

In [ ]:
mcp = mag.generate_mcp_script("news_processing_workflow")
# 保存并行执行脚本
with open("news_workflow_parallel.py", "w", encoding="utf-8") as f:
    f.write(mcp["parallel_script"])

# 保存顺序执行脚本
with open("news_workflow_sequential.py", "w", encoding="utf-8") as f:
    f.write(mcp["sequential_script"])

#### （7） 运行图

In [14]:
result = mag.run("deepresearch",input_text="深度探究谷歌开源Agent协议：A2A。以及探究微软2025build大会上，A2A的重要性。",parallel=False)
print(result)

{'graph_name': 'deepresearch', 'conversation_id': 'deepresearch_20250522_131229', 'input': '深度探究谷歌开源Agent协议：A2A。以及探究微软2025build大会上，A2A的重要性。', 'output': '# 深度研究报告\n\n## 问题分析\n### 问题分析报告：谷歌开源Agent协议A2A及微软2025 Build大会上的重要性  \n\n---\n\n#### **1. 核心问题识别**  \n用户的核心问题分为两个部分：  \n1. **技术层面**：深度探究谷歌开源的Agent协议A2A（Agent-to-Agent）的技术细节、设计目标、实现机制及其在开源生态中的定位。  \n2. **行业应用层面**：分析A2A协议在微软2025 Build大会上的重要性，包括微软如何可能集成或响应这一协议，以及其对多智能体系统（Multi-Agent Systems, MAS）生态的影响。  \n\n这两个问题的关联性在于：A2A作为一种开源协议，可能成为未来多智能体协作的标准，而微软作为谷歌的主要竞争对手，其Build大会的动向可能揭示行业对A2A的接受度或替代方案。  \n\n---\n\n#### **2. 关键概念提取**  \n- **A2A（Agent-to-Agent协议）**：  \n  - 定义：一种开源协议，旨在标准化智能体（Agent）之间的通信、协作和数据交换。  \n  - 可能特性：  \n    - 跨平台兼容性（支持不同框架的Agent交互）。  \n    - 安全性（如身份验证、数据加密）。  \n    - 可扩展性（支持动态加入/退出Agent）。  \n  - 相关技术：可能基于gRPC、HTTP/3、WebAssembly等现代协议或运行时。  \n\n- **微软2025 Build大会**：  \n  - 背景：微软年度开发者大会，通常发布重大技术战略（如Azure AI、Copilot生态）。  \n  - 潜在关联点：  \n    - 微软是否采纳A2A协议？  \n    - 是否推出竞争性协议（如基于OpenAI的Agent标准）？  \n    - 对多智能体协作的未来愿景（如与Azure云服务的整合）。

## 对话管理

#### (1)列出所有对话

In [ ]:
result = mag.list_conversations()
result

#### (2)获取对话结果

In [ ]:
result = mag.get_conversation('')
display(Markdown(result.get("output")))

In [ ]:
attactments = result.get("attachments")
print(json.dumps(attactments, indent=2, ensure_ascii=False))

In [ ]:
def display_attachments(attachments):
    """
    显示附件内容，支持md、html文件,这里只提供demo，运行时产生其他扩展文件请自行设计。或直接前往目录查看。
    使用notebook的markdown进行展示
    
    参数:
    attachments - 附件列表
    """
    if not attachments:
        print("没有可显示的附件")
        return
    
    for attachment in attachments:
        file_type = attachment["type"].lower()
        file_path = attachment["path"]
        file_name = attachment["name"]
        
        # 跳过不支持的文件类型
        if file_type not in ["md", "html", "docx", "pdf"]:
            continue
            
        print(f"\n📄 {file_name} ({file_type.upper()}):")
        
        try:
            # Markdown文件 - 使用IPython的Markdown显示
            if file_type == "md":
                with open(file_path, 'r', encoding='utf-8') as f:
                    md_content = f.read()
                display(Markdown(md_content))
                
            # HTML文件 - 使用IPython的HTML显示，或在浏览器中打开
            elif file_type == "html":
                try:
                    webbrowser.open(file_path)
                except:
                    print(f"已在浏览器中打开 HTML 文件")
                
        except Exception as e:
            print(f"无法显示文件 {file_name}: {str(e)}")

display_attachments(attactments)